In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import string
import unicodedata
from pathlib import Path
import collections

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading Data

In [3]:
# name data from: https://download.pytorch.org/tutorial/data.zip

In [4]:
all_chars = string.ascii_letters + " .,;'-"
print(all_chars)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'-


In [5]:
def sanitise_line(line):
    # https://stackoverflow.com/a/518232/5013267
    return ''.join(
        c for c in unicodedata.normalize('NFD', line)
        if unicodedata.category(c) != 'Mn'
        and c in all_chars
    )

In [6]:
name_data = {}
for path in Path("names").rglob("*.txt"):
    with open(path, "r", encoding="utf8") as f:
        names = []
        for line in f.read().strip().split("\n"):
            names.append(sanitise_line(line))
        name_data[path.stem] = names
    
    print(f"Loaded: {path}")

Loaded: names/Arabic.txt
Loaded: names/Chinese.txt
Loaded: names/Czech.txt
Loaded: names/Dutch.txt
Loaded: names/English.txt
Loaded: names/French.txt
Loaded: names/German.txt
Loaded: names/Greek.txt
Loaded: names/Irish.txt
Loaded: names/Italian.txt
Loaded: names/Japanese.txt
Loaded: names/Korean.txt
Loaded: names/Polish.txt
Loaded: names/Portuguese.txt
Loaded: names/Russian.txt
Loaded: names/Scottish.txt
Loaded: names/Spanish.txt
Loaded: names/Vietnamese.txt


# Data Preparation Stuff

In [7]:
class OneHotTranslator:
    def __init__(self, elements):
        self.elements = elements
        self.n_elements = len(self.elements)
        
    def index_to_vec(self, index):
        return F.one_hot(torch.tensor([index]), num_classes=self.n_elements)[0]
    
    def index_from_vec(self, vec):
        return vec.argmax()
    
    def elm_to_vec(self, elm):
        return self.index_to_vec(self.elements.index(elm))
    
    def elm_from_vec(self, vec):
        return self.elements[int(self.index_from_vec(vec))]
    
    def __len__(self):
        return self.n_elements
    
    def __getitem__(self, val):
        if isinstance(val, int):
            return self.index_to_vec(val)
        else:
            return self.elm_to_vec(val)

In [8]:
category_translator = OneHotTranslator(tuple(name_data.keys()))
print(category_translator.elements)
print("len:", len(category_translator))

('Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese')
len: 18


In [9]:
char_translator = OneHotTranslator(["<SOS>", "<EOS>"] + list(all_chars))
print(char_translator.elements)
print("len:", len(char_translator))

['<SOS>', '<EOS>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ', '.', ',', ';', "'", '-']
len: 60


In [10]:
# convenience functions
def build_name(name):
    """
    Build a list of one-hot encoded vectors representing the given name string
    """
    return [char_translator.elm_to_vec(char) for char in (["<SOS>"] + list(name) + ["<EOS>"])]

def unbuild_name(t_chars):
    """
    Inverse of build_name
    """
    return "".join([char_translator.elm_from_vec(vec) for vec in t_chars])

def build_category(category):
    """
    Build a one-hot encoded vector representing the given category string
    """
    return category_translator.elm_to_vec(category)

def unbuild_category(vec):
    """
    Inverse of build_category
    """
    return category_translator.elm_from_vec(vec)

In [11]:
# simple Dataset class wrapper around the above variables
class NamesDataset(Dataset):
    def __init__(self):
        self._data_pairs = [(cat, name) for cat in name_data for name in name_data[cat]]
    
    def __len__(self):
        return len(self._data_pairs)
    
    def __getitem__(self, index):
        cat, name = self._data_pairs[index]
        return build_category(cat), build_name(name)

# Neural

In [12]:
class SeqModel(nn.Module):
    def __init__(self, prime_size, input_size, hidden_size):
        super().__init__()
        self.prime_size = prime_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTMCell(self.input_size + self.prime_size, self.hidden_size)
        
        # NB: output size = input size
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        self.dropout = nn.Dropout(0.05)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, t_prime, t_input, t_hidden, t_cell):
        # t_prime:     batches of vectors for the LSTM to be primed on
        #              shape: (N, prime_size)
        #
        # t_input:     shape: (N, input_size)
        #              t_input[0]: one-hot vector encoding of a character
        #
        # t_hidden:    shape: (N, hidden_size)
        #
        # t_cell:      shape: (N, hidden_size)
        
        # lstm_input:  t_prime and t_input concated together
        #              shape: (N, prime_size + input_size)
        lstm_input = torch.cat((t_prime, t_input), dim=1)
        t_hidden, t_cell = self.lstm(lstm_input.float(), (t_hidden, t_cell))
        
        # t_output:    shape: (N, input_size)
        #              NB: t_output's shape = t_input's shape
        t_output = self.fc(self.dropout(t_hidden))
        t_output = self.softmax(t_output)
        
        return t_output, t_hidden, t_cell
    
    def init_hidden(self, batch_size):
        t_hidden = torch.zeros(batch_size, self.hidden_size)
        t_cell = torch.zeros(batch_size, self.hidden_size)
        
        return t_hidden, t_cell

In [13]:
model = SeqModel(len(category_translator), len(char_translator), 32)
model.to(device)

SeqModel(
  (lstm): LSTMCell(78, 32)
  (fc): Linear(in_features=32, out_features=60, bias=True)
  (dropout): Dropout(p=0.05, inplace=False)
  (softmax): LogSoftmax(dim=1)
)

In [14]:
test_category_index = 0

t_prime = category_translator[test_category_index].view(1, len(category_translator))
print("t_prime shape:\t", t_prime.shape)

t_input = char_translator["<SOS>"].view(1, len(char_translator))
print("t_input shape:\t", t_input.shape)

t_hidden, t_cell = model.init_hidden(1)
print("t_hidden shape:\t", t_hidden.shape)
print("t_cell shape:\t", t_cell.shape)

t_prime shape:	 torch.Size([1, 18])
t_input shape:	 torch.Size([1, 60])
t_hidden shape:	 torch.Size([1, 32])
t_cell shape:	 torch.Size([1, 32])


In [15]:
t_output, t_hidden, t_cell = model.forward(t_prime, t_input, t_hidden, t_cell)
print("t_output shape:\t", t_output.shape)
print("t_hidden shape:\t", t_hidden.shape)
print("t_cell shape:\t", t_cell.shape)

t_output shape:	 torch.Size([1, 60])
t_hidden shape:	 torch.Size([1, 32])
t_cell shape:	 torch.Size([1, 32])


# Training

In [16]:
# config
max_length = 30
batch_size = 64
num_epochs = 100
learning_rate = 0.0003
print_interval = 100

In [17]:
train_dataloader = DataLoader(NamesDataset(), batch_size=batch_size, shuffle=True, collate_fn=lambda x: x)

In [18]:
samples = next(iter(train_dataloader))

In [26]:
for cat, name in samples:
    print("({}, {})".format(unbuild_category(cat), unbuild_name(name)))

(Arabic, <SOS>Attia<EOS>)
(Arabic, <SOS>Gaber<EOS>)
(Russian, <SOS>Dzhikovich<EOS>)
(Russian, <SOS>Baidakoff<EOS>)
(Dutch, <SOS>Ramaaker<EOS>)
(Russian, <SOS>Tsei<EOS>)
(Russian, <SOS>Raisky<EOS>)
(Russian, <SOS>Gulai<EOS>)
(Russian, <SOS>Makhinya<EOS>)
(German, <SOS>Von essen<EOS>)
(Czech, <SOS>Kouba<EOS>)
(Arabic, <SOS>Tuma<EOS>)
(Dutch, <SOS>Tholberg<EOS>)
(Russian, <SOS>Lutchenko<EOS>)
(English, <SOS>Crook<EOS>)
(Portuguese, <SOS>Duarte<EOS>)
(Russian, <SOS>Rennenkampf<EOS>)
(English, <SOS>Hutchison<EOS>)
(Chinese, <SOS>Lim<EOS>)
(Russian, <SOS>Kartalov<EOS>)
(Russian, <SOS>Vyborny<EOS>)
(English, <SOS>Fields<EOS>)
(Arabic, <SOS>Ghannam<EOS>)
(Spanish, <SOS>Soto<EOS>)
(French, <SOS>Duguay<EOS>)
(Russian, <SOS>Lubutin<EOS>)
(Arabic, <SOS>Mifsud<EOS>)
(Arabic, <SOS>Antoun<EOS>)
(German, <SOS>Langer<EOS>)
(Russian, <SOS>Gets<EOS>)
(Russian, <SOS>Adibekov<EOS>)
(Russian, <SOS>Multyh<EOS>)
(English, <SOS>Panesar<EOS>)
(Czech, <SOS>Cablikova<EOS>)
(Russian, <SOS>Balawin<EOS>)
(Russian, <

In [49]:
torch.stack([samples[i][0] for i in range(batch_size)]).shape

torch.Size([64, 18])

In [20]:
criterion = nn.NLLLoss()
optimiser = optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
losses = collections.deque([], maxlen=print_interval)

In [53]:
for epoch_index in range(num_epochs):
    for batch_index, samples in enumerate(train_dataloader):
        model.zero_grad()
        
        cur_batch_size = len(samples)
        t_hidden, t_cell = map(lambda x: x.to(device), model.init_hidden(cur_batch_size))
        t_prime = torch.stack([samples[i][0] for i in range(cur_batch_size)]).to(device)

0
1
2
3
4


KeyboardInterrupt: 